In [1]:
from image_acquisition.image_sensor import *
from image_acquisition.playback_images import *
from image_acquisition.video_file_writer import *
#from image_acquisition.ueye_python_driver import UEyePythonDriver

from feature_computation.object_tracking import *
from feature_computation.object_detection_and_tracking import *
from feature_computation.visualization import *

import matplotlib as plt
import csv
from datetime import datetime

In [2]:
# Load the ONNX model

output_path = "../../../makerspace_20241126/output/object_tracking_20241127.mp4"
raw_output_path =  "../../../makerspace_20241126/output/object_tracking_raw_20241127.mp4"


onnx_model_path = "../../makerspace_20241126/export/onnx/simplified-model.onnx"
print(os.path.realpath(onnx_model_path))
video_path = "../../makerspace_20241126/videos/soldering_recordings/Stefan/rec_1.avi"
print(os.path.realpath(video_path))

trackers = {
   "pad": MultiObjectTracker(class_ids=[0,1,4,5], disappearance_threshold=-1, ratio_threshold=0.7),
   "cabel": MultiObjectTracker(class_ids=2, disappearance_threshold=50, ratio_threshold=0.7),
   "soldering_iron": KalmanFilterTracker(class_ids=3, ratio_threshold=0.7, disappearance_threshold=70)
}

C:\Users\ricar\Documents\Programming\makerspace_20241126\export\onnx\simplified-model.onnx
C:\Users\ricar\Documents\Programming\makerspace_20241126\videos\soldering_recordings\Stefan\rec_1.avi


In [3]:

visualizer = ObjectTrackerVisualization(trackers)

In [4]:
import state_update as su

pads = [su.pad(), su.pad()]

In [20]:
def float_state_to_string_state(float_state):
    if float_state == 0.0:
        return "presoldered"
    if float_state == 1.0:
        return "soldered"
    if float_state == 4.0:
        return "unsoldered"
    if float_state == 5.0:
        return "in_process"
    raise Exception("unknown pad state")

csv_file_name = "output.csv"
csv_file = open(csv_file_name, mode='w', newline='')
fieldnames = [
    "Time", 
    "pad_0_state", "pad_0_x", "pad_0_y", "pad_0_w", "pad_0_h",
    "pad_1_state", "pad_1_x", "pad_1_y", "pad_1_w", "pad_1_h",
    "cabel_0_x", "cabel_0_y", "cabel_0_w", "cabel_0_h",
    "cabel_1_x", "cabel_1_y", "cabel_1_w", "cabel_1_h", 
    "iron_0_x", "iron_0_y", "iron_0_w", "iron_0_h" 
]
csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
csv_writer.writeheader() 

try:
    with VideoFileImageSensor(video_path=video_path) as img_src:
        with video_file_writer_matching_image_sensor(img_src, output_path) as writer:
            with onnx_network_and_trackers(onnx_model_path, trackers.values()) as network_and_trackers:

                def callback(frame: np.ndarray):
                    network_and_trackers.update_trackers(frame)
                    disp_img = frame.copy()
                    visualizer.draw_visualization_on_frame(disp_img)

                    height, width = disp_img.shape[:2]
                    new_width, new_height = width // 3, height // 3
                    disp_img = cv2.resize(disp_img, (new_width, new_height), interpolation=cv2.INTER_AREA)
                    cv2.imshow("object tracking", disp_img)
                    writer.write(disp_img)

                    current_time = datetime.now().strftime("%H:%M:%S")
                    csv_row = {"Time": current_time}

                    detected_pads = trackers["pad"].get_objects()
                    for i in range(2): 
                        if i < len(detected_pads):
                            dp = detected_pads[i]
                            pads[dp.id].pad_pos = dp.bounding_box
                            pads[dp.id].pad_state = float_state_to_string_state(dp.state)

                            csv_row[f"pad_{i}_state"] = pads[dp.id].pad_state
                            csv_row[f"pad_{i}_x"] = int(pads[dp.id].pad_pos.x.item()) if pads[dp.id].pad_pos else None
                            csv_row[f"pad_{i}_y"] = int(pads[dp.id].pad_pos.y.item()) if pads[dp.id].pad_pos else None
                            csv_row[f"pad_{i}_w"] = int(pads[dp.id].pad_pos.w.item()) if pads[dp.id].pad_pos else None
                            csv_row[f"pad_{i}_h"] = int(pads[dp.id].pad_pos.h.item()) if pads[dp.id].pad_pos else None

                            print(f"state of pad {i}: {pads[dp.id].pad_state}")
                            #print(f"state of pad {i}: {pads[dp.id].state}")
                        else:
                            csv_row[f"pad_{i}_state"] = "no pad detected"
                            csv_row[f"pad_{i}_x"] = None
                            csv_row[f"pad_{i}_y"] = None
                            csv_row[f"pad_{i}_w"] = None
                            csv_row[f"pad_{i}_h"] = None

                    su.wire_pos = [w.bounding_box for w in trackers["cabel"].get_objects()]
                    for i in range(2): 
                        if i < len(su.wire_pos):
                            wire = su.wire_pos[i]
                            csv_row[f"cabel_{i}_x"] = int(wire.x.item())
                            csv_row[f"cabel_{i}_y"] = int(wire.y.item())
                            csv_row[f"cabel_{i}_w"] = int(wire.w.item())
                            csv_row[f"cabel_{i}_h"] = int(wire.h.item())
                            print(f"Cable {i} location: {wire}")
                        else:
                            csv_row[f"cabel_{i}_x"] = 0
                            csv_row[f"cabel_{i}_y"] = 0
                            csv_row[f"cabel_{i}_w"] = 0
                            csv_row[f"cabel_{i}_h"] = 0

                    su.iron_pos = None
                    for si in trackers["soldering_iron"].get_objects():
                        su.iron_pos = si.bounding_box
                        #print(type(su.iron_pos))
                        #print(vars(su.iron_pos))
                        #print(su.iron_pos)
                        #iron = su.iron_pos[i]
                        csv_row[f"iron_0_x"] = int(su.iron_pos.x.item())
                        csv_row[f"iron_0_y"] = int(su.iron_pos.y.item())
                        csv_row[f"iron_0_w"] = int(su.iron_pos.w.item())
                        csv_row[f"iron_0_h"] = int(su.iron_pos.h.item())
                        print(f"Iron 0 location: {su.iron_pos}")

                    csv_writer.writerow(csv_row)

                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        raise NoMoreImages

                run_playback(img_src, callback)
finally:
    csv_file.close()
    cv2.destroyAllWindows()


state of pad 0: unsoldered
state of pad 1: unsoldered
Iron 0 location: BoundingBox(x=array([1639.4061], dtype=float32), y=array([1236.4851], dtype=float32), w=array([210.099], dtype=float32), h=array([188.2185], dtype=float32))
state of pad 0: unsoldered
state of pad 1: unsoldered
Iron 0 location: BoundingBox(x=array([1640.2021], dtype=float32), y=array([1242.4442], dtype=float32), w=array([209.86206], dtype=float32), h=array([187.71948], dtype=float32))
state of pad 0: unsoldered
state of pad 1: unsoldered
Iron 0 location: BoundingBox(x=array([1637.3698], dtype=float32), y=array([1247.2444], dtype=float32), w=array([215.79321], dtype=float32), h=array([190.22632], dtype=float32))
state of pad 0: unsoldered
state of pad 1: unsoldered
Iron 0 location: BoundingBox(x=array([1637.3419], dtype=float32), y=array([1247.2914], dtype=float32), w=array([215.85144], dtype=float32), h=array([190.25098], dtype=float32))
state of pad 0: unsoldered
state of pad 1: unsoldered
Iron 0 location: Bounding